<a href="https://colab.research.google.com/github/SmartPracticeschool/SBSPS-Challenge-3626-Solace-Food-Demand-Prediction/blob/master/notebooks/IBM_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start#
Data is imported into the notebook from their corresponding .csv files. The required python modules and packages are imported.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root = 'gdrive/My Drive/IBM Hack/data/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import sklearn
import numpy as np
import pickle


The 3 different dataframes are merged into one to perform further operations and derive inferences. Since the no. of columns of all 3 dataframes are same, they are merged without any shape changes.

In [ ]:
df_raw=pd.read_csv(f"{root}df_raw.csv",)


In [ ]:
df_test = pd.read_csv(f"{root}test.csv")

In [ ]:
df_raw.drop(columns=["Unnamed: 0"],inplace=True)

In [ ]:
df_trn=df_raw.copy()

The combined dataframe is then viewed to see all the columns and it's shape.

In [ ]:
def to_cat(df,cols):
  mapping={}
  for col in cols:
    mapping[col]={}
    df_cat=pd.DataFrame(df[col].astype('category'))
    unq=df[col].unique()
    for val in unq:
      mapping[col][val]=df_cat.loc[df_cat[col]==val,col].cat.codes.iloc[0]
    df.loc[:,col]=df[col].astype('category').cat.codes
  return mapping

In [ ]:
mapping=to_cat(df_trn,["category","cuisine","center_type"])

In [ ]:
#df_trn=pd.concat([df_trn,a,b,c],axis=1)

In [ ]:
def apply_cats(df, mapping):
  for column, map in mapping.items():
    for cats, vals in map.items():
      df.loc[df[column]==cats,column]=vals
    df[column]= df[column].astype('int16')



In [ ]:
test_df=pd.read_csv(f"{root}test.csv")
meal=pd.read_csv(f"{root}meal_info.csv")
center=pd.read_csv(f"{root}fulfilment_center_info.csv")

In [ ]:
test_raw=pd.merge(test_df,meal,on='meal_id')

In [ ]:
df_test=pd.merge(test_raw,center,on='center_id')

In [ ]:
test_df

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0
...,...,...,...,...,...,...,...,...
32568,1250239,155,61,1543,482.09,484.09,0,0
32569,1039516,155,61,2304,483.09,483.09,0,0
32570,1158107,155,61,2664,322.07,323.07,0,0
32571,1444235,155,61,2569,322.07,323.07,0,0


In [ ]:
test_df = test_df.merge(meal, on='meal_id')
test_df = test_df.merge(center, on='center_id')

test_df

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area
0,1028232,146,55,1885,158.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0
1,1262649,147,55,1885,159.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0
2,1453211,149,55,1885,157.14,158.14,0,0,Beverages,Thai,647,56,TYPE_C,2.0
3,1262599,150,55,1885,159.14,157.14,0,0,Beverages,Thai,647,56,TYPE_C,2.0
4,1495848,151,55,1885,160.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1412025,146,61,2104,583.03,581.03,0,0,Fish,Continental,473,77,TYPE_A,4.5
32569,1287019,147,61,2104,582.03,582.03,0,1,Fish,Continental,473,77,TYPE_A,4.5
32570,1396176,149,61,2104,629.53,629.53,0,0,Fish,Continental,473,77,TYPE_A,4.5
32571,1331977,150,61,2104,629.53,629.53,0,0,Fish,Continental,473,77,TYPE_A,4.5


In [ ]:
test_df['discount'] = (test_df.base_price - test_df.checkout_price)*100/test_df.base_price

In [ ]:
test_df

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area,discount
0,1028232,146,55,1885,158.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0,0.628496
1,1262649,147,55,1885,159.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0,0.000000
2,1453211,149,55,1885,157.14,158.14,0,0,Beverages,Thai,647,56,TYPE_C,2.0,0.632351
3,1262599,150,55,1885,159.14,157.14,0,0,Beverages,Thai,647,56,TYPE_C,2.0,-1.272750
4,1495848,151,55,1885,160.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0,-0.628496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1412025,146,61,2104,583.03,581.03,0,0,Fish,Continental,473,77,TYPE_A,4.5,-0.344216
32569,1287019,147,61,2104,582.03,582.03,0,1,Fish,Continental,473,77,TYPE_A,4.5,0.000000
32570,1396176,149,61,2104,629.53,629.53,0,0,Fish,Continental,473,77,TYPE_A,4.5,0.000000
32571,1331977,150,61,2104,629.53,629.53,0,0,Fish,Continental,473,77,TYPE_A,4.5,0.000000


In [ ]:
apply_cats(test_df, mapping)

In [ ]:
pickle.dump(mapping,open(f"{root}mapping", "wb"))

In [ ]:
df_trn.loc[:,"discount"]=(df_trn.base_price-df_trn.checkout_price)*100/df_trn.base_price

The categories are then label encoded to perform different operations on the data.

In [ ]:
#df_trn(columns=['category','cuisine','center_type'],inplace=True)


In [ ]:
df_raw

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1,1018704,2,55,1885,135.83,152.29,0,0,323,Beverages,Thai,647,56,TYPE_C,2.0
2,1196273,3,55,1885,132.92,133.92,0,0,96,Beverages,Thai,647,56,TYPE_C,2.0
3,1116527,4,55,1885,135.86,134.86,0,0,163,Beverages,Thai,647,56,TYPE_C,2.0
4,1343872,5,55,1885,146.50,147.50,0,0,215,Beverages,Thai,647,56,TYPE_C,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1437396,141,61,2104,583.03,630.53,0,1,13,Fish,Continental,473,77,TYPE_A,4.5
456544,1060716,142,61,2104,581.03,582.03,0,0,42,Fish,Continental,473,77,TYPE_A,4.5
456545,1029231,143,61,2104,583.03,581.03,0,0,40,Fish,Continental,473,77,TYPE_A,4.5
456546,1141147,144,61,2104,582.03,581.03,0,0,53,Fish,Continental,473,77,TYPE_A,4.5


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_trn.drop(['num_orders','id','week','base_price'],axis=1), df_trn.num_orders, test_size=0.2,random_state=42)

The dataset is then split into train and test data with a specific weightage. This helps in improving the accuracy of the model on the real test dataset because we are able to derive inferences about the model from the mock test dataset which helps in twitching the model for better performance.

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm

In [ ]:
X_train.columns

Index(['center_id', 'meal_id', 'checkout_price', 'emailer_for_promotion',
       'homepage_featured', 'category', 'cuisine', 'city_code', 'region_code',
       'center_type', 'op_area', 'discount'],
      dtype='object')

In [ ]:
#model1=BaggingRegressor(LinearRegression(),n_estimators=100,max_samples=0.5,max_features=0.5)

In [ ]:
#model1.fit(X_train,y_train)

In [ ]:
#model1.score(X_train,y_train)

In [ ]:
#df_trn=pd.concat([df_trn,df_raw["checkout_price"]],axis=1)

In [ ]:
val_score=[]
test_score=[]

In [ ]:
from xgboost import XGBRegressor, DMatrix

In [ ]:
model=XGBRegressor(max_depth=20,n_estimators=100, n_jobs=-1,)

In [ ]:
model.fit(X_train, y_train)

[12:42:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
model.score(X_train,y_train)

0.9792475499717707

In [ ]:
model.score(X_test,y_test)

0.8180018880856174

In [ ]:
test_df.category.dtype

dtype('int16')

In [ ]:
X_train.columns

Index(['center_id', 'meal_id', 'checkout_price', 'emailer_for_promotion',
       'homepage_featured', 'category', 'cuisine', 'city_code', 'region_code',
       'center_type', 'op_area', 'discount'],
      dtype='object')

In [ ]:
model.predict(test_df.drop(columns=['id','week','base_price']))

array([148.38667 , 170.39786 , 137.33165 , ...,  18.411667,  18.411667,
        27.050468], dtype=float32)

In [ ]:
#import pickle

In [ ]:
model.base_score

0.5

In [ ]:
test_df.to_csv(f"{root}nayat.csv",index=False)

In [ ]:
#pickle.dump(model,f"{root}model.pickle")

In [ ]:
#with open(f"{root}model.pickle", 'wb') as pickle_file:
    pickle.dump(model, pickle_file)

IndentationError: ignored

In [ ]:
model.save_model(f"{root}model")

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
(mean_absolute_error(model.predict(X_train),y_train))

In [ ]:
(mean_absolute_error(model.predict(X_test),y_test))

In [ ]:
diff=(np.subtract(model.predict(X_test),y_test))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(20,20))
sns.distplot(diff,)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.concat([X_test,pd.DataFrame(model.predict(X_test))],axis=1).corr(),cmap="BrBG",annot=True)

In [ ]:
import matplotlib.pyplot as plt
import xgboost

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_trn.corr(),annot=True,cmap="BrBG")

In [ ]:
plt.figure(figsize=(30,30))
xgboost.plot_importance(model,importance_type="weight")

In [ ]:
plt.figure(figsize=(30,10))
plt.bar(*zip(*feat_imps.items()))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
def best_ntree_score(estimator, X, y):
    """
    This scorer uses the best_ntree_limit to return
    the best AUC ROC score
    """
    try:
        y_predict = estimator.predict(X,ntree_limit=estimator.best_ntree_limit)
    except AttributeError:
        y_predict = estimator.predict(X)
    return roc_auc_score(y, y_predict[:, 1])

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat = pd.DataFrame(yhat)
yhat.shape

In [ ]:
X_test.reset_index(inplace=True)
X_test.rename(columns={"index":"row_id"},inplace=True)
X_test


In [ ]:
X_test.shape

In [ ]:
yhat.shape

In [ ]:

prediction = pd.concat([X_test,yhat], axis=1, ignore_index=False)
prediction.rename(columns={'0':'pred'},inplace=True)


In [ ]:
prediction.rename(columns={'Unnamed: 0' : 'prediction'},inplace =True)

In [ ]:
prediction[prediction.columns[-1]]

In [ ]:
intpred = pd.DataFrame(prediction[prediction.columns[-1]].round())
intpred

In [ ]:
prediction['intpred'] = intpred
prediction

In [ ]:
import xgboost
xgboost.plot_importance(model)


In [ ]:
newdf = df_raw.drop(columns=['cuisine','center_type'])
newdf

In [ ]:
newX_train, newX_test, newy_train, newy_test = train_test_split(newdf.drop(['num_orders','id','week'],axis=1), newdf.num_orders, test_size=0.2,shuffle=False)

In [ ]:
newmodel=XGBRegressor(max_depth=10,n_estimators=250, n_jobs=-1,subsample=0.8,eta=0.2)

In [ ]:
newmodel.fit(newX_train, newy_train)

In [ ]:
newmodel.score(newX_test,newy_test)

In [ ]:
xgboost.plot_importance(newmodel)